In [ ]:
! pip3 install onnxoptimizer==0.2.6
! pip list | grep onnx

In [186]:
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.transformation.infer_shapes import InferShapes
from qonnx.transformation.general import GiveReadableTensorNames, GiveUniqueNodeNames
from qonnx.transformation.infer_data_layouts import InferDataLayouts
from qonnx.transformation.infer_datatypes import InferDataTypes
from qonnx.transformation.merge_onnx_models import MergeONNXModels
from onnx.helper import make_graph, make_model, set_model_props
from onnx import helper, TensorProto
import qonnx.core.data_layout as DataLayout
from qonnx.core.datatype import DataType
import numpy as np
import onnx

def get_init(model, tensor_name):
    for init in model.graph.initializer:
        if tensor_name == init.name:
            init_val = init.raw_data
            model.graph.initializer.remove(init)
            break
    for inp in model.graph.input:
        if tensor_name in inp.name:
            model.graph.input.remove(inp)
    
    return np.frombuffer(init_val, dtype = np.int64)

def add_attr(model, op_type):
    new_node = []
    for ind,n in enumerate(model.graph.node):
        new_attr = {}
        if op_type in n.name:        
            if len(n.input) > 1:
                new_attr["starts"] = get_init(model,n.input[1])
                new_attr["ends"] = get_init(model,n.input[2])
                new_attr["axes"] = get_init(model,n.input[3])
            if len(n.input[1:]) >3:
                new_attr["steps"] = get_init(model,n.input[4])
            if not new_attr == {}:
                new_node = make_node(model,new_attr,op_type)
                model.graph.node.remove(n)
                model.graph.node.append(new_node)
    
    return model

def make_node(model,slice_attr,op_type):
    for n in model.graph.node:
        if op_type in n.name:
            node = helper.make_node(
                op_type,
                name = n.name,
                inputs=[n.input[0]],
                outputs=[n.output[0]],
                **slice_attr
            )
    
            return node
    
def new_model(model):
    """
    Overwrites the main opset in an ONNX file.
    Does not change any node definition.
    :param model: ONNX model
    :param new_opset: new opset
    :return: ONNX model
    """    
    
    model_config = {}
    model_config["opset_imports"] = [onnx.helper.make_operatorsetid("",9)]
    
    graph = make_graph(
        model.graph.node, model.graph.name, model.graph.input,
        model.graph.output, model.graph.initializer)
    onnx_model = make_model(graph, functions=model.functions,**model_config)
    onnx_model.ir_version = 4 #model.ir_version
    onnx_model.producer_name = model.producer_name
    onnx_model.producer_version = model.producer_version
    onnx_model.domain = model.domain
    onnx_model.model_version = model.model_version
    onnx_model.doc_string = model.doc_string
    if len(model.metadata_props) > 0:  # pragma: no cover
        values = {p.key: p.value for p in model.metadata_props}
        set_model_props(onnx_model, values)

    return ModelWrapper(onnx_model)

def cnt_ops(model):
    nodes = []
    for n in model.graph.node:
        nodes.append(n.op_type)
    ops,cnts = np.unique(np.array(nodes),return_counts=True)
    total = {}
    for ind,k in enumerate(ops):
        total[str(k)] = cnts[ind]

    return total

In [191]:
def pre_process(model):
    
    global_in = model.graph.input[0]
    idt = model.get_tensor_datatype(global_in.name)

    node = helper.make_node(
        "Div",
        inputs = [global_in.name,"b"],
        outputs = ["y"],
        name = "Div_node"
    )
    model.graph.node.append(node)
    model.graph.node[0].input[0] = "y"
    model.set_initializer('b',np.array(255,dtype = np.float32))
#     model.set_tensor_datatype("y",DataType[])
    model.set_tensor_datatype(model.graph.input[0].name,DataType["UINT8"])
    model.set_tensor_layout(model.graph.input[0].name,DataLayout.NCHW)    
    
    return model

In [192]:
model = ModelWrapper('tinyyolo-20210831.onnx')

model = add_attr(model, "Slice")
model = new_model(model.model)
model = pre_process(model)

model.save("tinyyolo-20210831_updated.onnx")
model = model.transform(InferShapes())
print("Saved!")


/workspace/qonnx/src/qonnx/core/modelwrapper.py:93: UserWarning: Some old-style domain attributes were automatically converted to new-style,
                i.e. domain=finn to domain=qonnx.custom_op.<general|fpgadataflow|...>
  warnings.warn(


Saved!


In [193]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil

model_file = "tinyyolo-20210831_updated"
onnx_model = "%s.onnx" %model_file
print(onnx_model)
final_output_dir = "build_output_%s"%model_file

#Delete previous run results if exist
if os.path.exists(final_output_dir):
    shutil.rmtree(final_output_dir)
    print("Previous run results deleted!")

cfg = build.DataflowBuildConfig(
    output_dir          = final_output_dir,
    mvau_wwidth_max     = 80,
    target_fps          = 1000000,
    synth_clk_period_ns = 10.0,
    board               = "KV260_SOM",
    shell_flow_type     = build_cfg.ShellFlowType.VIVADO_ZYNQ,
    generate_outputs=[
        build_cfg.DataflowOutputType.BITFILE,
        build_cfg.DataflowOutputType.PYNQ_DRIVER,
        build_cfg.DataflowOutputType.DEPLOYMENT_PACKAGE,
    ]
)

build.build_dataflow_cfg(onnx_model, cfg)

tinyyolo-20210831_updated.onnx


In [194]:
import qonnx.util.basic as util
print(onnx_model)
model = ModelWrapper(onnx_model)
model = model.transform(InferShapes())
model = model.transform(InferDataTypes())
model = model.transform(GiveReadableTensorNames())

model.save("ReadableNames.onnx")


tinyyolo-20210831_updated.onnx
